### Configuration Spark

In [1]:
import pyspark
pyspark.__version__

'3.3.0'

In [2]:
from pyspark.sql import SparkSession, functions as f
from pyspark import SparkFiles
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
from pyspark.ml import Pipeline
import plotly.express as px
import plotly.graph_objects as go

In [3]:
# create section
spark_session = SparkSession.builder.appName('PySpark Recomendation').getOrCreate()

22/09/22 14:58:06 WARN Utils: Your hostname, ronald-MpRonald resolves to a loopback address: 127.0.1.1; using 192.168.1.100 instead (on interface enp1s0)
22/09/22 14:58:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/22 14:58:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Loading Dataset

In [4]:
url_data = 'https://github.com/MpRonald/datasets/blob/main/dados_musicas.csv?raw=true'
spark_session.sparkContext.addFile(url_data)
path_data = 'file://' + SparkFiles.get('dados_musicas.csv')
data = spark_session.read.csv(path_data, header=True, sep=';', inferSchema=True)
data.limit(20).toPandas()

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,speechiness,tempo,artists_song
0,0.285,2000,0.002390,Coldplay,0.429,266773,0.661,0,3AJwUDP919kvQ9QcozQPxg,0.000121,11,0.2340,-7.227,1,Yellow,84,0.0281,173.372,Coldplay - Yellow
1,0.613,2000,0.143000,OutKast,0.843,270507,0.806,1,0I3q5fE6wg7LIfHGngUTnV,0.000000,4,0.0771,-5.946,0,Ms. Jackson,80,0.2690,94.948,OutKast - Ms. Jackson
2,0.400,2000,0.009580,Linkin Park,0.556,216880,0.864,0,60a0Rd6pjrkxjPbaKzXjfq,0.000000,3,0.2090,-5.870,0,In the End,84,0.0584,105.143,Linkin Park - In the End
3,0.543,2000,0.006640,3 Doors Down,0.545,233933,0.865,0,6ZOBP3NvffbU4SZcrnt1k6,0.000011,11,0.1680,-5.708,0,Kryptonite,78,0.0286,99.009,3 Doors Down - Kryptonite
4,0.760,2000,0.030200,Eminem,0.949,284200,0.661,1,3yfqSUWxFvZELEM4PmlwIR,0.000000,5,0.0454,-4.244,0,The Real Slim Shady,80,0.0572,104.504,Eminem - The Real Slim Shady
5,0.941,2000,0.000996,Disturbed,0.695,279213,0.876,1,40rvBMQizxkIqnjPdEWY1v,0.000007,3,0.1060,-4.262,0,Down with the Sickness,76,0.0553,89.954,Disturbed - Down with the Sickness
6,0.722,2000,0.061600,Nelly,0.850,291782,0.700,1,3Gf5nttwcX9aaSQXRWidEZ,0.000002,7,0.2440,-6.490,1,Ride Wit Me,74,0.0478,101.875,Nelly - Ride Wit Me
7,0.507,2000,0.037100,Eminem,0.780,404107,0.768,1,3UmaczJpikHgJFyBTAJVoz,0.000002,6,0.5180,-4.325,0,Stan,78,0.2380,80.063,Eminem - Stan
8,0.861,2000,0.031000,*NSYNC,0.610,200400,0.926,0,62bOmKYxYg7dhrC6gH9vFn,0.001200,8,0.0821,-4.843,0,Bye Bye Bye,71,0.0479,172.638,*NSYNC - Bye Bye Bye
9,0.894,2000,0.300000,Britney Spears,0.751,211160,0.834,0,6naxalmIoLFWR0siv8dnQQ,0.000018,1,0.3550,-5.444,0,Oops!...I Did It Again,78,0.0437,95.053,Britney Spears - Oops!...I Did It Again


In [5]:
data.show(vertical=True, truncate=False)

-RECORD 0--------------------------------------------------------------------
 valence          | 0.285                                                    
 year             | 2000                                                     
 acousticness     | 0.00239                                                  
 artists          | Coldplay                                                 
 danceability     | 0.429                                                    
 duration_ms      | 266773                                                   
 energy           | 0.6609999999999999                                       
 explicit         | 0                                                        
 id               | 3AJwUDP919kvQ9QcozQPxg                                   
 instrumentalness | 1.21E-4                                                  
 key              | 11                                                       
 liveness         | 0.234                                       

In [6]:
data.printSchema()

root
 |-- valence: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- artists: string (nullable = true)
 |-- danceability: double (nullable = true)
 |-- duration_ms: integer (nullable = true)
 |-- energy: double (nullable = true)
 |-- explicit: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- key: integer (nullable = true)
 |-- liveness: double (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- artists_song: string (nullable = true)



In [7]:
# shape
data.count(), len(data.columns)

(20311, 19)

In [8]:
# checking null values
data.select([f.count(f.when(f.isnull(c), 1)).alias(c) for c in data.columns]).show()

+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+-----------+-----+------------+
|valence|year|acousticness|artists|danceability|duration_ms|energy|explicit| id|instrumentalness|key|liveness|loudness|mode|name|popularity|speechiness|tempo|artists_song|
+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+-----------+-----+------------+
|      0|   0|           0|      0|           0|          0|     0|       0|  0|               0|  0|       0|       0|   0|   0|         0|          0|    0|           0|
+-------+----+------------+-------+------------+-----------+------+--------+---+----------------+---+--------+--------+----+----+----------+-----------+-----+------------+



In [9]:
# checking year
print(sorted(data.select('year').distinct().collect()))

[Row(year=2000), Row(year=2001), Row(year=2002), Row(year=2003), Row(year=2004), Row(year=2005), Row(year=2006), Row(year=2007), Row(year=2008), Row(year=2009), Row(year=2010), Row(year=2011), Row(year=2012), Row(year=2013), Row(year=2014), Row(year=2015), Row(year=2016), Row(year=2017), Row(year=2018), Row(year=2019), Row(year=2020)]


### Analysing Dataset

In [10]:
url_data_years = 'https://github.com/MpRonald/datasets/raw/main/data_by_year.csv'
spark_session.sparkContext.addFile(url_data_years)
path_data_years = 'file://' + SparkFiles.get('data_by_year.csv')
data_year = spark_session.read.csv(path_data_years, header=True, sep=',', inferSchema=True)
data_year.limit(10).toPandas()

,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,1921,0.886896,0.418597,260537.166667,0.231815,0.344878,0.205710,-17.048667,0.073662,101.531493,0.379327,0.653333,2
1,1,1922,0.938592,0.482042,165469.746479,0.237815,0.434195,0.240720,-19.275282,0.116655,100.884521,0.535549,0.140845,10
2,1,1923,0.957247,0.577341,177942.362162,0.262406,0.371733,0.227462,-14.129211,0.093949,114.010730,0.625492,5.389189,0
3,1,1924,0.940200,0.549894,191046.707627,0.344347,0.581701,0.235219,-14.231343,0.092089,120.689572,0.663725,0.661017,10
4,1,1925,0.962607,0.573863,184986.924460,0.278594,0.418297,0.237668,-14.146414,0.111918,115.521921,0.621929,2.604317,5
5,1,1926,0.660817,0.599880,156881.657475,0.211467,0.333093,0.232370,-18.492538,0.483704,109.648033,0.436910,1.422351,9
6,1,1927,0.936179,0.648268,184993.598374,0.264321,0.391328,0.168450,-14.422374,0.113610,114.846524,0.659700,0.801626,7
7,1,1928,0.938617,0.534288,214827.906423,0.207948,0.494835,0.175289,-17.191983,0.159911,106.772262,0.495713,1.525773,1
8,1,1929,0.601427,0.647670,168999.412815,0.241801,0.215204,0.236000,-16.530376,0.490001,110.948357,0.636530,0.340336,7
9,1,1930,0.936715,0.518176,195150.285343,0.333524,0.352206,0.221311,-12.869221,0.119910,109.871194,0.616238,0.926715,2


In [11]:
data_year_2000 = data_year.filter('year >= 2000')
data_year_2000.limit(10).toPandas()

,mode,year,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,2000,0.289323,0.590918,242724.642638,0.625413,0.101168,0.197686,-8.247766,0.089205,118.999323,0.559475,46.684049,7
1,1,2001,0.286842,0.583318,240307.796010,0.626986,0.107214,0.187026,-8.305095,0.089182,117.765399,0.541479,48.750125,7
2,1,2002,0.282624,0.576160,239503.283000,0.641270,0.088048,0.193911,-7.686640,0.084308,119.239738,0.542397,48.655500,7
3,1,2003,0.256471,0.575763,244670.575230,0.660165,0.083049,0.196976,-7.485545,0.093926,120.914622,0.530504,48.626407,7
4,1,2004,0.280559,0.567680,237378.708037,0.648868,0.077934,0.202199,-7.601655,0.094239,121.290346,0.524489,49.273143,7
5,1,2005,0.255764,0.572281,237229.588205,0.653209,0.090194,0.190082,-7.466159,0.093334,121.617967,0.532531,50.953333,0
6,1,2006,0.279986,0.568230,234042.914359,0.650326,0.077701,0.188289,-7.265501,0.085847,121.798615,0.520028,51.313846,7
7,1,2007,0.254081,0.563414,241049.962564,0.668305,0.072957,0.196127,-7.044536,0.084347,124.087516,0.516794,51.075897,7
8,1,2008,0.249192,0.579193,240107.315601,0.671461,0.063662,0.198431,-6.843804,0.077356,123.509934,0.527542,50.630179,0
9,1,2009,0.261929,0.564190,238140.013265,0.670749,0.075872,0.205252,-7.046015,0.085458,123.463808,0.507170,51.440816,0


In [12]:
data_year_2000.count()

21

In [13]:
len(data_year_2000.columns)

14

In [14]:
# changing dataset toPandas
fig = px.line(data_year_2000.toPandas(), x='year', y='loudness', markers=True,
              title='Loudness variation by YEARS').show()

In [15]:
temp = data_year_2000.toPandas()
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['year'], y=temp['acousticness'], name='Acousticness'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['valence'], name='Valence'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['danceability'], name='Danciability'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['energy'], name='Energy'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['instrumentalness'], name='Instrumentalness'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['liveness'], name='Liveness'))
fig.add_trace(go.Scatter(x=temp['year'], y=temp['speechiness'], name='Speechiness')).show()

### Correlation Matrix

In [16]:
fig = px.imshow(data_year_2000.drop('mode').toPandas().corr(), text_auto=True).show()

In [17]:
url_data_genre = 'https://github.com/MpRonald/datasets/raw/main/dados_musicas_genero.csv'
spark_session.sparkContext.addFile(url_data_genre)
path_data_genre = 'file://' + SparkFiles.get('dados_musicas_genero.csv')
data_genre = spark_session.read.csv(path_data_genre, header=True, inferSchema=True)
data_genre.limit(10).toPandas()

,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.361600,-31.514333,0.040567,75.336500,0.103783,27.833333,6
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.500000,5
2,1,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.126000,-9.180000,0.047000,133.444000,0.975000,48.000000,7
3,1,[],0.651417,0.529093,2.328809e+05,0.419146,0.205309,0.218696,-12.288965,0.107872,112.857352,0.513604,20.859882,7
4,1,a cappella,0.676557,0.538961,1.906285e+05,0.316434,0.003003,0.172254,-12.479387,0.082851,112.110362,0.448249,45.820071,7
5,1,abstract,0.459210,0.516167,3.431965e+05,0.442417,0.849667,0.118067,-15.472083,0.046517,127.885750,0.307325,43.500000,1
6,1,abstract beats,0.342147,0.623000,2.299362e+05,0.527800,0.333603,0.099653,-7.918000,0.116373,112.413800,0.493507,58.933333,10
7,1,abstract hip hop,0.243854,0.694571,2.318492e+05,0.646235,0.024231,0.168543,-7.349328,0.214258,108.244987,0.571391,39.790702,2
8,0,accordeon,0.323000,0.588000,1.640000e+05,0.392000,0.441000,0.079400,-14.899000,0.072700,109.131000,0.709000,39.000000,2
9,1,accordion,0.446125,0.624812,1.670616e+05,0.373437,0.193738,0.160300,-14.487063,0.078538,112.872438,0.658688,21.937500,2


In [18]:
data_genre.count(), data_genre.select('genres').distinct().count()

(2973, 2973)

In [19]:
# shape
data_genre.count(), len(data_genre.columns)

(2973, 14)

### PCA and Standard Scale

In [20]:
X = data_genre.columns
X

['mode',
 'genres',
 'acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'instrumentalness',
 'liveness',
 'loudness',
 'speechiness',
 'tempo',
 'valence',
 'popularity',
 'key']

In [21]:
X.remove('genres')
X

['mode',
 'acousticness',
 'danceability',
 'duration_ms',
 'energy',
 'instrumentalness',
 'liveness',
 'loudness',
 'speechiness',
 'tempo',
 'valence',
 'popularity',
 'key']

In [22]:
data_genre_vector = VectorAssembler(inputCols=X, outputCol='features')\
                                    .transform(data_genre)\
                                    .select(['features', 'genres'])

In [23]:
data_genre_vector.toPandas()

22/09/22 14:59:17 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,features,genres
0,"[1.0, 0.9793333333333332, 0.16288333333333335,...",21st century classical
1,"[1.0, 0.49478, 0.2993333333333333, 1048887.333...",432hz
2,"[1.0, 0.762, 0.7120000000000001, 115177.0, 0.8...",8-bit
3,"[1.0, 0.6514170195595453, 0.5290925603549332, ...",[]
4,"[1.0, 0.676557304985755, 0.5389612464387464, 1...",a cappella
...,...,...
2968,"[1.0, 0.22262496820571934, 0.547081530483321, ...",zolo
2969,"[0.0, 0.161, 0.863, 206320.0, 0.909, 0.0, 0.10...",zouglou
2970,"[1.0, 0.2632611111111112, 0.7488888888888888, ...",zouk
2971,"[0.0, 0.993, 0.7056666666666666, 198417.333333...",zurich indie


##### Standard Scale

In [24]:
scaler = StandardScaler(inputCol='features', outputCol='scaled_features')
scaler_model = scaler.fit(data_genre_vector)
data_genre_scaler = scaler_model.transform(data_genre_vector)
data_genre_scaler.toPandas()

,features,genres,scaled_features
0,"[1.0, 0.9793333333333332, 0.16288333333333335,...",21st century classical,"[2.6817483100027903, 3.06270998538775, 1.08107..."
1,"[1.0, 0.49478, 0.2993333333333333, 1048887.333...",432hz,"[2.6817483100027903, 1.5473461333255456, 1.986..."
2,"[1.0, 0.762, 0.7120000000000001, 115177.0, 0.8...",8-bit,"[2.6817483100027903, 2.3830343861798493, 4.725..."
3,"[1.0, 0.6514170195595453, 0.5290925603549332, ...",[],"[2.6817483100027903, 2.0372036185737374, 3.511..."
4,"[1.0, 0.676557304985755, 0.5389612464387464, 1...",a cappella,"[2.6817483100027903, 2.1158258818926794, 3.577..."
...,...,...,...
2968,"[1.0, 0.22262496820571934, 0.547081530483321, ...",zolo,"[2.6817483100027903, 0.6962243496803475, 3.631..."
2969,"[0.0, 0.161, 0.863, 206320.0, 0.909, 0.0, 0.10...",zouglou,"[0.0, 0.5035020159776322, 5.727836235891636, 2..."
2970,"[1.0, 0.2632611111111112, 0.7488888888888888, ...",zouk,"[2.6817483100027903, 0.8233074544904095, 4.970..."
2971,"[0.0, 0.993, 0.7056666666666666, 198417.333333...",zurich indie,"[0.0, 3.1054503221477563, 4.683595717026879, 2..."


##### PCA

In [25]:
pca = PCA(k=2, inputCol='scaled_features', outputCol='pca_features')
model_pca = pca.fit(data_genre_scaler)
data_genre_pca = model_pca.transform(data_genre_scaler)

22/09/22 14:59:25 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/09/22 14:59:25 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


22/09/22 14:59:27 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
22/09/22 14:59:27 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [26]:
data_genre_pca.toPandas()

,features,genres,scaled_features,pca_features
0,"[1.0, 0.9793333333333332, 0.16288333333333335,...",21st century classical,"[2.6817483100027903, 3.06270998538775, 1.08107...","[2.507095366888567, 0.4381691373769795]"
1,"[1.0, 0.49478, 0.2993333333333333, 1048887.333...",432hz,"[2.6817483100027903, 1.5473461333255456, 1.986...","[-0.5969679056633482, 4.981612052751348]"
2,"[1.0, 0.762, 0.7120000000000001, 115177.0, 0.8...",8-bit,"[2.6817483100027903, 2.3830343861798493, 4.725...","[-4.158460276223561, -0.836652508107994]"
3,"[1.0, 0.6514170195595453, 0.5290925603549332, ...",[],"[2.6817483100027903, 2.0372036185737374, 3.511...","[-2.387344878512217, -0.4877989015663402]"
4,"[1.0, 0.676557304985755, 0.5389612464387464, 1...",a cappella,"[2.6817483100027903, 2.1158258818926794, 3.577...","[-2.6501218371679087, -0.5756819768820469]"
...,...,...,...,...
2968,"[1.0, 0.22262496820571934, 0.547081530483321, ...",zolo,"[2.6817483100027903, 0.6962243496803475, 3.631...","[-3.9181421935898793, 0.416507018372443]"
2969,"[0.0, 0.161, 0.863, 206320.0, 0.909, 0.0, 0.10...",zouglou,"[0.0, 0.5035020159776322, 5.727836235891636, 2...","[-6.549271970898396, -0.668292822368372]"
2970,"[1.0, 0.2632611111111112, 0.7488888888888888, ...",zouk,"[2.6817483100027903, 0.8233074544904095, 4.970...","[-4.192043469635213, -0.6687762062702765]"
2971,"[0.0, 0.993, 0.7056666666666666, 198417.333333...",zurich indie,"[0.0, 3.1054503221477563, 4.683595717026879, 2...","[-1.7680020287988045, -2.969740528792513]"


In [27]:
data_genre_pca.select('pca_features').toPandas()

,pca_features
0,"[2.507095366888567, 0.4381691373769795]"
1,"[-0.5969679056633482, 4.981612052751348]"
2,"[-4.158460276223561, -0.836652508107994]"
3,"[-2.387344878512217, -0.4877989015663402]"
4,"[-2.6501218371679087, -0.5756819768820469]"
...,...
2968,"[-3.9181421935898793, 0.416507018372443]"
2969,"[-6.549271970898396, -0.668292822368372]"
2970,"[-4.192043469635213, -0.6687762062702765]"
2971,"[-1.7680020287988045, -2.969740528792513]"


### Pipeline

In [35]:
pca_pipeline = Pipeline(
    stages=[VectorAssembler(inputCols=X, outputCol='features'),
    StandardScaler(inputCol='features', outputCol='scaled_features'),
    PCA(k=2, inputCol='scaled_features', outputCol='pca_features')])

# adjust pipeline
pca_pipeline_model = pca_pipeline.fit(data_genre)

# transform
data_genre_pca = pca_pipeline_model.transform(data_genre)

data_genre_pca.limit(10).toPandas()

,mode,genres,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,popularity,key,features,scaled_features,pca_features
0,1,21st century classical,0.979333,0.162883,1.602977e+05,0.071317,0.606834,0.361600,-31.514333,0.040567,75.336500,0.103783,27.833333,6,"[1.0, 0.9793333333333332, 0.16288333333333335,...","[2.6817483100027903, 3.06270998538775, 1.08107...","[2.507095366888567, 0.4381691373769795]"
1,1,432hz,0.494780,0.299333,1.048887e+06,0.450678,0.477762,0.131000,-16.854000,0.076817,120.285667,0.221750,52.500000,5,"[1.0, 0.49478, 0.2993333333333333, 1048887.333...","[2.6817483100027903, 1.5473461333255456, 1.986...","[-0.5969679056633482, 4.981612052751348]"
2,1,8-bit,0.762000,0.712000,1.151770e+05,0.818000,0.876000,0.126000,-9.180000,0.047000,133.444000,0.975000,48.000000,7,"[1.0, 0.762, 0.7120000000000001, 115177.0, 0.8...","[2.6817483100027903, 2.3830343861798493, 4.725...","[-4.158460276223561, -0.836652508107994]"
3,1,[],0.651417,0.529093,2.328809e+05,0.419146,0.205309,0.218696,-12.288965,0.107872,112.857352,0.513604,20.859882,7,"[1.0, 0.6514170195595453, 0.5290925603549332, ...","[2.6817483100027903, 2.0372036185737374, 3.511...","[-2.387344878512217, -0.4877989015663402]"
4,1,a cappella,0.676557,0.538961,1.906285e+05,0.316434,0.003003,0.172254,-12.479387,0.082851,112.110362,0.448249,45.820071,7,"[1.0, 0.676557304985755, 0.5389612464387464, 1...","[2.6817483100027903, 2.1158258818926794, 3.577...","[-2.6501218371679087, -0.5756819768820469]"
5,1,abstract,0.459210,0.516167,3.431965e+05,0.442417,0.849667,0.118067,-15.472083,0.046517,127.885750,0.307325,43.500000,1,"[1.0, 0.45921, 0.5161666666666667, 343196.5, 0...","[2.6817483100027903, 1.4361065885533446, 3.425...","[-1.4965091203367626, 1.8644183183717793]"
6,1,abstract beats,0.342147,0.623000,2.299362e+05,0.527800,0.333603,0.099653,-7.918000,0.116373,112.413800,0.493507,58.933333,10,"[1.0, 0.3421466666666667, 0.623, 229936.2, 0.5...","[2.6817483100027903, 1.0700095430229417, 4.134...","[-3.9235207721573238, 0.2851835002352837]"
7,1,abstract hip hop,0.243854,0.694571,2.318492e+05,0.646235,0.024231,0.168543,-7.349328,0.214258,108.244987,0.571391,39.790702,2,"[1.0, 0.2438540633608816, 0.6945709366391184, ...","[2.6817483100027903, 0.762614984512678, 4.6099...","[-4.611011109831115, -0.6783790472312371]"
8,0,accordeon,0.323000,0.588000,1.640000e+05,0.392000,0.441000,0.079400,-14.899000,0.072700,109.131000,0.709000,39.000000,2,"[0.0, 0.32299999999999995, 0.588, 164000.0, 0....","[0.0, 1.010131373669411, 3.902627701858959, 1....","[-2.8376900630842297, -0.5712993716580516]"
9,1,accordion,0.446125,0.624812,1.670616e+05,0.373437,0.193738,0.160300,-14.487063,0.078538,112.872438,0.658688,21.937500,2,"[1.0, 0.446125, 0.6248125, 167061.5625, 0.3734...","[2.6817483100027903, 1.395185322223734, 4.1469...","[-2.706690139892783, -1.2593788079708297]"


In [36]:
data_genre_pca.select('pca_features').toPandas()

,pca_features
0,"[2.507095366888567, 0.4381691373769795]"
1,"[-0.5969679056633482, 4.981612052751348]"
2,"[-4.158460276223561, -0.836652508107994]"
3,"[-2.387344878512217, -0.4877989015663402]"
4,"[-2.6501218371679087, -0.5756819768820469]"
...,...
2968,"[-3.9181421935898793, 0.416507018372443]"
2969,"[-6.549271970898396, -0.668292822368372]"
2970,"[-4.192043469635213, -0.6687762062702765]"
2971,"[-1.7680020287988045, -2.969740528792513]"
